In [27]:
from checkreq import FindImport, FindreqLibraries, changereqfile

In [11]:
import os

def GetFilesInDirectory(directory):
    paths = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            paths.append(file_path)
    # print(paths)
    return paths


In [18]:
list_of_libs = []

for dir in GetFilesInDirectory('src'):
    list_of_libs += FindImport(dir)

In [19]:
list_of_libs

['datetime',
 'asyncio',
 'base64',
 'requests',
 'time',
 'langchain',
 'json',
 'boto3']

In [28]:
for dir in GetFilesInDirectory('src'):
    changereqfile("requirements2.txt", 'requirements.txt', dir)
    print(FindreqLibraries("requirements.txt", dir))

['requests @ file:///private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_b143dxenr7/croot/requests_1716902835977/work', 'requests-file @ file:///Users/ktietz/demo/mc3/conda-bld/requests-file_1629455781986/work', 'requests-oauthlib==1.3.1', 'requests-toolbelt @ file:///Users/builder/cbouss/perseverance-python-buildout/croot/requests-toolbelt_1699238632371/work']
['langchain==0.1.16', 'langchain-aws==0.1.2', 'langchain-cohere==0.1.5', 'langchain-community==0.0.34', 'langchain-core==0.1.46', 'langchain-openai==0.0.2.post1', 'langchain-text-splitters==0.0.1', 'json5 @ file:///tmp/build/80754af9/json5_1624432770122/work', 'jsonpatch @ file:///private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_3ajyoz8zoj/croot/jsonpatch_1714483362270/work', 'jsonpointer==2.1', 'jsonref==1.1.0', 'jsonschema @ file:///Users/builder/cbouss/perseverance-python-buildout/croot/jsonschema_1701803145350/work', 'jsonschema-specifications @ file:///Users/builder/cbouss/perseverance-python-buildout/cro

In [1]:
import dspy
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from typing import Annotated, Any, Dict, Optional, Sequence, TypedDict, List, Tuple

/opt/anaconda3/envs/storm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Uses aws credentials from aws cli
bedrock = dspy.Bedrock(region_name="us-east-1")

model_kwargs =  { 
        "max_tokens": 4096,
        "temperature": 0.4,
        "top_k": 50,
        "top_p": 0.9
    }

lm = dspy.AWSAnthropic(bedrock, "anthropic.claude-3-5-sonnet-20240620-v1:0", **model_kwargs)

In [3]:
dspy.configure(lm=lm)

In [4]:
## Get input fields through LLM again

In [5]:
class Input(BaseModel):
    python_version: str = Field(description="Python version to be used")
    type_of_application: str = Field(description="Type of python application")
    work_directory: str = Field(description="Working directory of the application")
    entrypoint_filename: str = Field(description="File name for starting the application endpoint")

class Output(BaseModel):
    dockerfile: str = Field(description="The complete dockerfile code")


class CreateDockerFile(dspy.Signature):
    """I need a Dockerfile for a basic Streamlit app. The app is written in Python."""

    input: Input = dspy.InputField()
    output: Output = dspy.OutputField()


In [6]:
predictor = dspy.TypedPredictor(CreateDockerFile)
input = Input(
    python_version = "3.11",
    type_of_application = 'Streamlit',
    work_directory = 'src',
    entrypoint_filename = 'app.py'
)
mapped_parameters = predictor(input=input)

In [12]:
print(mapped_parameters.output.dockerfile)

# Use an official Python runtime as a parent image
FROM python:3.11-slim

# Set the working directory in the container
WORKDIR /app

# Copy the current directory contents into the container at /app
COPY ./src /app

# Install any needed packages specified in requirements.txt
COPY requirements.txt /app/
RUN pip install --no-cache-dir -r requirements.txt

# Make port 8501 available to the world outside this container
EXPOSE 8501

# Run app.py when the container launches
CMD ["streamlit", "run", "app.py", "--server.port=8501", "--server.address=0.0.0.0"]


In [14]:
with open("Dockerfile", "w") as file:
    file.write(mapped_parameters.output.dockerfile)